# Predicting Movie Review Sentiment with [kpe/bert-for-tf2](https://github.com/kpe/bert-for-tf2)
## Serve with Clipper & Benchmark Part


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=benchmark&ea=bentoml-benchmark&dt=bert-movie-review-serve-clipper)

In [1]:
# Paste the bundle_path generated by `pack_movie_reviews.ipynb` here
bentoml_bundle_path = '/home/bentoml/bentoml/repository/MovieReviewService/20200508142510_3CD595'

In [2]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())

try:
    cl.start_clipper(cache_size=1)
except:
    cl.connect()

APP_NAME = bentoml_bundle_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'strings', 'default_pred', 10 * 1000 * 1000)

20-05-08:14:44:09 INFO     [__init__.py:118] No local BentoML config file found, using default configurations
20-05-08:14:44:10 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-05-08:14:44:12 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp69gmju3q.yml
20-05-08:14:44:13 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-05-08:14:44:13 INFO     [clipper_admin.py:236] [default-cluster] Application 20200508142510_3cd595 was successfully registered


In [5]:
clipper_model_name, clipper_model_version = deploy_bentoml(cl, bentoml_bundle_path, 'predict_clipper')
cl.set_num_replicas(clipper_model_name, 1)

[2020-05-08 15:07:39,855] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-08 15:07:39,869] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+196.g54dcd0e
[2020-05-08 15:07:40,867] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will 

20-05-08:15:07:43 ERROR    [clipper_admin.py:719] [default-cluster] Received error status code: 400 and message: model with name 'moviereviewservice-predict-clipper' and version '20200508142510-3cd595' already exists


ClipperException: Received error status code: 400 and message: model with name 'moviereviewservice-predict-clipper' and version '20200508142510-3cd595' already exists

In [6]:
try:
    cl.link_model_to_app(APP_NAME, clipper_model_name)
except:
    pass
addr = cl.get_query_addr()
server_url = f"http://{addr}/{APP_NAME}/predict"
print(clipper_model_name)
server_url

20-05-08:15:07:54 INFO     [clipper_admin.py:303] [default-cluster] Model moviereviewservice-predict-clipper is now linked to application 20200508142510_3cd595


moviereviewservice-predict-clipper


'http://localhost:1337/20200508142510_3cd595/predict'

In [1]:
server_url = 'http://localhost:1337/20200508142510_3cd595/predict'
import random
import json

def get_request_producer():

    url = server_url
    method = "POST"
    headers = {"content-type": "application/json"}
    pred_sentence =  "The acting was a bit lacking."

    def _gen_data(size=1):
        raw = pred_sentence
        noise = f" By User:{random.random()}"
        data = json.dumps(
               {"input": raw + noise}  # random offset to avoid cache
        )
        return url, method, headers, data

    return _gen_data

# test
get_request_producer()()

('http://localhost:1337/20200508142510_3cd595/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"input": "The acting was a bit lacking. By User:0.21683649807156957"}')

In [2]:
%%time
import requests
gen = get_request_producer()
url, me, h, d = gen()
r = requests.request(me, url, headers=h, data=d).json()
print(r)


def verify_result(status, msg):
    if status // 100 == 2 and "default_pred" not in msg:
        return True
    return False

{'query_id': 48294, 'output': 'negative', 'default': False}
CPU times: user 78.4 ms, sys: 7.98 ms, total: 86.4 ms
Wall time: 5.19 s


In [3]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, verify_response=verify_result, timeout=10)
b.start_session(60, total_user=900, spawn_speed=100)

======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧═══════